In [1]:
import flywheel
import datetime
import pytz
import time 

# Create client
fw = flywheel.Client()

# define containers & files
gear = fw.lookup('gears/presurg/1.4.0_0.4.0')
project = fw.lookup('davis/presurgicalEpilepsy')

In [2]:
# create subject list
fin = 'patients.txt'
sid_list = [line.rstrip('\n').split()[0] for line in open(fin)]

In [4]:
for sub in sid_list:
    ses = fw.lookup('davis/presurgicalEpilepsy/%s/ses-01' % (sub))
    print('Trying subject %s' % (fw.get(ses.parents['subject']).label))

    # Check if there is task-fMRI data
    bold_img = None
    for acq in ses.acquisitions():
        if 'BOLD_Sentence' in acq.label or 'BOLD_Wordgen' in acq.label or 'BOLD_SceneMem' in acq.label:
            bold_container=fw.get(acq.id)
            for file in bold_container.files:
                if '.nii.gz' in file.name:
                    bold_img=file
    if bold_img is None:
        print("  %s: No task-fMRI data." % (sub))

    # Get the fmriprep analysis and check if already been run successfully
    for analysis in ses.analyses:
        if analysis.files and "fmriprep" in analysis.label:
            fprep = fw.get(analysis.id)
            print(analysis.label)
    print("Using %s" % (fprep.label))
    for file in fprep.files:
        if "fmriprep" in file.name:
            fmriprepdir = file
            #print(' Using %s as input fmriprep directory' % (fmriprepdir.name))

    alreadydone = False
    for analysis in ses.analyses:
        if analysis.files and 'sens8' in analysis.label:
            print(analysis.label)
            alreadydone = True
            print("already done " + sub)

    # Run gear
    if fprep is not None and bold_img is not None and not alreadydone:
        print(' Running analysis for %s' % (sub))
        inputs = {'fmriprepdir': fmriprepdir }
        config = {'AROMA': True, 'light_output': False,'fwhm': 8, 'cluster_size_thresh': 150, 'alpha': 0.10}
        analysis_id = gear.run(analysis_label='lingua-map a=0.10', config=config, inputs=inputs, destination=ses)

Trying subject P01
  P01: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P02
  P02: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P03
  P03: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P04
  P04: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P05
  P05: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P06
  P06: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P07
  P07: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P08
  P08: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P09
  P09: No task-fMRI data.
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
Trying subject P10
  P10: No task-fMR

Trying subject P75
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
 Running analysis for P75
Trying subject P76
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
 Running analysis for P76
Trying subject P77
SB_fmriprep 8-13-2019 14:30
Using SB_fmriprep 8-13-2019 14:30
 Running analysis for P77
Trying subject P78
WT_fmriprep
fmriprep 08-01-2019 15:30:05
SB_fmriprep 8-13-2019 15:21
Using SB_fmriprep 8-13-2019 15:21
 Running analysis for P78
Trying subject P79
SB_fmriprep 08-20-2019 13:24:00
WT_fmriprep
WT_fmriprep
Using WT_fmriprep
 Running analysis for P79
Trying subject P80
SB_fmriprep 08-20-2019 13:24:39
Using SB_fmriprep 08-20-2019 13:24:39
 Running analysis for P80


In [ ]:
# sleep for an hour
#time.sleep(216000)

In [10]:
utc = pytz.UTC
threshold = datetime.datetime(2019, 11, 6, 10)
threshold = utc.localize(threshold)

# Retry failed analyses
for sub in sid_list[28:]:
    ses = fw.lookup('davis/presurgicalEpilepsy/%s/ses-01' % (sub))
    for analysis in ses.analyses:
        if "lingua-map roi" in analysis.label and analysis.job.state=='failed' and (analysis.created > threshold):
            print("%s's job failed. Retrying..." % (sub))
            job = analysis.job
            print(job.attempt)
            print(job.failure_reason)
            new_id = fw.retry_job(job.id)
            print(new_id)


P33's job failed. Retrying...
3
None
{'_id': '5de9607940116e0058c51b89'}
